## BPETokenzier 

### components

- vocab_init
  - special_tokens

- pre_tokenization(multiprocess)
  - pre_tokenize_worker
  - get_frequency_table {(l,o,w): 5 ...}

- merge
  - get_successive_pairs {lo: 7, ow: 7, we: 8, er: 2, wi: 3, id: 3, de: 3, es: 9, st: 9, ne: 6, ew: 6}.
  - get_lexicographically_greater_pair
  - merge_pre_tokens {(l,o,w): 5, (l,o,w,e,r): 2, (w,i,d,e,st): 3, (n,e,w,e,st): 6}


- Level 0
  - tokenize_word: word -> char list
  - count_pairs: 统计token列表中相邻对
  - merge_pair_in_word: 合并指定的pair

In [5]:
def tokenize_word(word: str) -> list[str]: 
    assert word
    return [char for char in word]

In [7]:
# 简单测试
assert tokenize_word("hello") == ['h', 'e', 'l', 'l', 'o']
assert tokenize_word("a") == ['a']
print("tokenize_word 测试通过！")

tokenize_word 测试通过！


In [10]:
from collections import defaultdict

def count_pairs(token_list: list[str]) -> dict[tuple[str, str], int]:
    pair_freq = defaultdict(int)
    for i in range(0, len(token_list) - 1):
        token_pair = (token_list[i], token_list[i+1])
        pair_freq[token_pair] += 1
    
    return pair_freq


In [12]:
assert count_pairs(['h', 'e', 'l', 'l', 'o']) == {('h','e'): 1, ('e','l'): 1, ('l','l'): 1, ('l','o'): 1}
print('Pass')

Pass


In [16]:
def merge_pair_in_word(word: list[str], pair: tuple[str, str]):
    new_word = []
    i = 0
    while i < len(word):
        if i < len(word) - 1 and (word[i], word[i+1]) == pair:
            new_word.append(word[i] + word[i+1])
            i += 2 
        else:
            new_word.append(word[i])
            i += 1
    return new_word

In [17]:
# 测试1：基础合并
assert merge_pair_in_word(['h', 'e', 'l', 'l', 'o'], ('l', 'l')) == ['h', 'e', 'll', 'o']

# 测试2：多次出现
assert merge_pair_in_word(['a', 'b', 'a', 'b', 'c'], ('a', 'b')) == ['ab', 'ab', 'c']

# 测试3：没有匹配
assert merge_pair_in_word(['h', 'e', 'l', 'o'], ('l', 'l')) == ['h', 'e', 'l', 'o']

### Simulate

In [35]:
# 维护tokenized_word
tokenized_word_freq = dict()

# 模拟数据
words = ["hello", "world", "hello"]
word_freq = defaultdict(int)
for word in words:
    word_freq[word] += 1

tokenized_word_freq_d = defaultdict(int)
for word, count in word_freq.items():
    tokenized_word = tokenize_word(word)
    tokenized_word_freq_d[tuple(tokenized_word)] = count

tokenized_word_freq = dict(tokenized_word_freq_d)

pair_freq = defaultdict(int)
for tokenized_word, count in tokenized_word_freq.items():
    part_pair_freq =  count_pairs(tokenized_word)
    for pf, freq in part_pair_freq.items():
        pair_freq[pf] += freq * count

sorted(dict(pair_freq).items(), key = lambda x: (x[1], x[0][0], x[0][1]), reverse=True)

[(('l', 'o'), 2),
 (('l', 'l'), 2),
 (('h', 'e'), 2),
 (('e', 'l'), 2),
 (('w', 'o'), 1),
 (('r', 'l'), 1),
 (('o', 'r'), 1),
 (('l', 'd'), 1)]